# Big Data for Engineers – Moodle - Solutions

# Spring 2023 -  Week 11 - Rumble



# Preparation for the moodle exercise in Spark


## 1. Install Java and Download Rumble
Please follow the instructions in Exercise11_Rumble_2023.ipynb.

## 2. Setup Working Environment
1. Go to the folder where this notebook is in: `cd [path to the folder]`
2. Start your jupyter notebook: `jupyter notebook`
3. Start your own Rumble server: `java -jar rumbledb-1.21.0-standalone.jar --server yes --port 9090`
4. Execute the cell below:

In [ ]:
!pip install rumbledb
%load_ext rumbledb
%env RUMBLEDB_SERVER=http://localhost:9090/jsoniq

Verify that jsoniq works.

In [2]:
%%jsoniq
1+1

Took: 0.0179288387298584 ms
2


## 3. Prepare the Data: The Great Language Game

This week you will be using again the [language confusion dataset](https://quietlyamused.org/blog/2014/03/12/language-confusion/). Make sure you use the correct dataset.
### Please use the first 80k records of the "confusion-100000.json" dataset
We can obtain the first 80k records like this: `subsequence(json-file("confusion-100000.json", 10), 1, 80000)`

In [3]:
%%jsoniq
count(subsequence(json-file("confusion-100000.json", 10), 1, 80000))

Took: 0.22565031051635742 ms
80000


## 4. Instructions
You should write __JSONiq__ queries only. \
For each quation, you have to submit the following results to the Moodle quiz:
- The query you wrote
- The query output __in the required format__ (which you will be graded on)
- The time it took you to run it (optional and not graded)

The execution time of the queries will be reported by Rumble.

# Questions

## Assignment 1
Find all games such that the guessed language is correct (=target), and such that this language is Spanish. What is the length of the resulting sequence?

In [4]:
%%jsoniq
count(
    for $i in subsequence(json-file("confusion-100000.json", 10), 1, 80000)
    where $i.guess eq $i.target and $i.target eq "Spanish"
    return $i
)

Took: 0.3441760540008545 ms
1672


## Assignment 2
Find the number of all distinct values of the guessed languages (i.e. the guess field). What is the length of the resulting sequence?

In [5]:
%%jsoniq
count(distinct-values(
    for $i in subsequence(json-file("confusion-100000.json", 10), 1, 80000)
    return $i.guess
))

Took: 0.40917491912841797 ms
68


## Assignment 3
Return the top three games where the guessed language is incorrect ($\neq$ target) ordered by country (ascending), then target language (ascending), then date (ascending). What is the sample id of the 3rd item in the list?

Enter it without quotes, for example 48f9c924e0d98c959d8a6f1862b3ce9a

In [6]:
%%jsoniq
let $ordered_confusion := for $i in subsequence(json-file("confusion-100000.json", 10), 1, 80000)
                where $i.guess ne $i.target
                order by $i.country, $i.target, $i.date
                return $i
for $j in $ordered_confusion
count $c
where $c eq 3
return $j.sample

Took: 1.2181332111358643 ms
"3520fdcdf2658685928fc37e72ab2134"


## Assignment 4
Aggregate all games by guessed and target language, counting the number of guessing games that were done for each pair (guess, target). How many times has Dutch been mistaken for Norwegian (i.e. Dutch was the true answer)?

In [7]:
%%jsoniq
count(
    for $i in subsequence(json-file("confusion-100000.json", 10), 1, 80000)
    where $i.guess eq "Norwegian" and $i.target eq "Dutch"
    return $i
)

Took: 0.2951669692993164 ms
19


## Assignment 5
Among all the games where the guess was correct (=target), what is the percentage of cases where the second choice (among the array of possible answers) was the target?

Please write the fraction rounding to 4 decimals (eg. 0.3323)

In [8]:
%%jsoniq
let $correct := for $i in subsequence(json-file("confusion-100000.json", 10), 1, 80000)
                where $i.guess eq $i.target
                return $i
let $second_correct := for $i in $correct
                where $i.target eq $i.choices[[2]]
                return $i
return count($second_correct) div count($correct) 

Took: 0.6639571189880371 ms
0.3752740329


## Assignment 6
For each target language, compute the percentage of successful guess games (i.e. guess == target) relative to all games for that target language, and display the pairs (target_language, percentage) in descending order of the percentage. What is the third language in this list?

In [9]:
%%jsoniq
let $grouped_data := for $i in subsequence(json-file("confusion-100000.json", 10), 1, 80000)
                    group by $target := $i.target
                    return {"target": $target, "guesses": $i.guess}
for $j in $grouped_data
let $correct := for $guess in $j.guesses[]
                where $guess eq $j.target
                return $guess
let $percentage := count($correct) div count($j.guesses[])
order by $percentage descending
count $c
where $c eq 3
return $j.target

Took: 2.6805953979492188 ms
"Spanish"


In [10]:
%%jsoniq
let $grouped_guesses := for $i in subsequence(json-file("confusion-100000.json", 10), 1, 80000)
                        group by $target := $i.target
                        return {"target": $target, "count": count($i.guess)}
let $correct_guesses := for $i in subsequence(json-file("confusion-100000.json", 10), 1, 80000)
                        where $i.guess eq $i.target
                        group by $target := $i.target
                        return {"target": $target, "count": count($i.guess)}
for $i in $grouped_guesses, $j in $correct_guesses
where $i.target eq $j.target
let $percentage := $j.count div $i.count
order by $percentage descending
count $c
where $c eq 3
return $i.target

Took: 11.627791166305542 ms
"Spanish"


## Assignment 7
How many games in France (country=FR) were played on the last day?

In [11]:
%%jsoniq
count(
    for $i in subsequence(json-file("confusion-100000.json", 10), 1, 80000)
    where $i.country eq "FR"
    group by $date := $i.date
    order by $date descending
    count $c
    where $c eq 1
    return $i
)

Took: 2.669163942337036 ms
265
